In [8]:
"""
Что делать?

Датасет ml-latest
Вспомнить подходы, которые мы разбирали
Выбрать понравившийся подход к гибридным системам
Написать свою
"""

from surprise import SVD, SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

import pandas as pd
import numpy as np


In [9]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [10]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [11]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [12]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [17]:
# объединим фреймы фильмов и рейтингов, применив метод  .join(). 
# Для этого поднимем индекс по полю movieId и следом вернем его из индекса
# в финале преобразований удалим пустые значения и перепишем сет
movies_and_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_and_ratings.dropna(inplace=True)

In [20]:
# Объявим переменную класса Reader и преобразуем датафрейм Pandas в датасет surprise 
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(movies_and_ratings[['userId', 'title', 'rating']], reader)

In [21]:
# Разобьем датасет surprise на трейн и тест 
trainset, testset = train_test_split(data, test_size=0.10)

In [24]:
# создадим модель, основанную на принципах коллаборативной фильтрации, SVD.
# Она позволяет произвести матричное разложение исходного массива,
# в результате которого получается произведение двух матриц. Здесь использованы следубщие параметры: 
# n_factors - количество факторов. По умолчанию 100.
# n_epochs - номер итерации процедуры SGD. По умолчанию 20.
# lr_all - Скорость обучения по всем параметрам. По умолчанию 0.005.

algo_svd = SVD(n_factors=25, n_epochs=18, lr_all=0.0005)
algo_svd.fit(trainset)

In [25]:
# теперь спрогнозируем значение и выясним ошибку 
# качество высокое - настолько, что есть опасность переобучения модели 
test_pred = algo_svd.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9010


0.9010046821986117

In [31]:
# Выберем наугад пользователя с его просмотренными фильмами
user_ = movies_and_ratings[movies_and_ratings.userId == 537].title.unique()
print('Просмотренные фильмы:\n', user_)
user_data = movies_and_ratings[movies_and_ratings.userId == 537]
user_data.head()

Просмотренные фильмы:
 ['Shawshank Redemption, The (1994)' 'Mask, The (1994)'
 "Schindler's List (1993)" 'Titanic (1997)' 'American History X (1998)'
 'Matrix, The (1999)' 'Green Mile, The (1999)'
 "Ocean's Eleven (a.k.a. Ocean's 11) (1960)" "Ocean's Eleven (2001)"
 'Lord of the Rings: The Fellowship of the Ring, The (2001)'
 'Lord of the Rings: The Two Towers, The (2002)'
 'Catch Me If You Can (2002)'
 'Pirates of the Caribbean: The Curse of the Black Pearl (2003)'
 'Scary Movie 3 (2003)'
 'Lord of the Rings: The Return of the King, The (2003)'
 'Notebook, The (2004)' "Ocean's Twelve (2004)"
 "Pirates of the Caribbean: Dead Man's Chest (2006)"
 'Departed, The (2006)' "Pirates of the Caribbean: At World's End (2007)"
 "Ocean's Thirteen (2007)" 'American Gangster (2007)'
 'Lions For Lambs (2007)' 'Bank Job, The (2008)' 'Dark Knight, The (2008)'
 'Hangover, The (2009)' 'Hurt Locker, The (2008)'
 'Harry Potter and the Half-Blood Prince (2009)' 'Avatar (2009)'
 'Inception (2010)' 'Social N

,movieId,title,genres,userId,rating,timestamp
8932,318,"Shawshank Redemption, The (1994)",Crime|Drama,537.0,4.0,1.424138e+09
10840,367,"Mask, The (1994)",Action|Comedy|Crime|Fantasy,537.0,4.0,1.424317e+09
14296,527,Schindler's List (1993),Drama|War,537.0,1.0,1.424142e+09
35110,1721,Titanic (1997),Drama|Romance,537.0,3.0,1.424142e+09
42235,2329,American History X (1998),Crime|Drama,537.0,4.5,1.424141e+09


In [37]:
# попробуем предсказать для конкретного пользователя оценку, которую он бы поставил фильму Mask, The (1994)
# Как видим, есть разница между реальной оценкой и спрогнозированной,
# однако последняя (примерно 3,5) не столь уж сильно отклонилась от фактического значения 
"""
здесь следующая логика: научившись данным методом предсказывать оценку фильма,
можем подбирать картины и по спрогнозированным высоким рейтингам от пользователя предлагать ему к просмотру новые фильмы 
"""
# algo_svd.predict(uid=537.0, iid='Mortal Kombat (1995)').est
algo_svd.predict(uid=537.0, iid='Mask, The (1994)').est

3.469716171335181

In [39]:
# теперь сделаем следующее: подбираемся ближе собственно к рекомендательной системе 
# зная, что интересует пользователя, предложим фильмы, которые могут ему понравиться
"""
логика такова: анализируем наивысшие оценки для фильмов, смотрим, что это за картины 
(например, смотрим на основе анализа по жанрам)
и следом подбираем фильмы с похожим жанровым составом
"""

current_user_id = 537.0
user_movies = movies_and_ratings[movies_and_ratings.userId == current_user_id].title.unique()

scores = []
titles = []

for movie in movies_and_ratings.title.unique():
    if movie in user_movies:
        continue
        
    scores.append(algo_svd.predict(uid=current_user_id, iid=movie).est)
    titles.append(movie)

In [44]:
# Выберем наивысших 10 оценок, которые пользователь оставил определенным фильмам (отсоритрованы по возрастанию)
sorted(scores)[-10:]

[4.119559334215643,
 4.123033472937851,
 4.182739820144523,
 4.212831899115976,
 4.237743939427499,
 4.261049737363572,
 4.264326543671754,
 4.287741961653223,
 4.322149041558788,
 4.326847825926272]

In [58]:
sort_film = sorted(titles)[-10:]
sort_film

['Zulu (2013)',
 '[REC] (2007)',
 '[REC]² (2009)',
 '[REC]³ 3 Génesis (2012)',
 'anohana: The Flower We Saw That Day - The Movie (2013)',
 'eXistenZ (1999)',
 'xXx (2002)',
 'xXx: State of the Union (2005)',
 '¡Three Amigos! (1986)',
 'À nous la liberté (Freedom for Us) (1931)']

In [63]:
movies_and_ratings[movies_and_ratings.title == 'Zulu (2013)']

,movieId,title,genres,userId,rating,timestamp
97713,116207,Zulu (2013),Crime|Drama|Thriller,448.0,1.5,1.461357e+09


In [48]:
# можно добавить третий метод:
# обработаем строчку жанров, чтбы затем на ее основе подобрать близкие фильмы по жанрам - их также можно рекомендовать
# объявим функцию для работы с жанрами, которая заменяет представление их описания на слова через пробелы 
# создадим переменную, прогнав строки поля жанров через функцию
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

movie_genres = [change_string(g) for g in movies.genres.values]

In [49]:
# Теперь представим фильм в виде вектора, который обработаем трансформером tfidf для придания разных весов
# Подключим метод ближайших соседей 

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

neigh = NearestNeighbors(n_neighbors=25, algorithm='ball_tree', n_jobs=-1, metric='manhattan') 
neigh.fit(X_train_tfidf)

C:\Users\zlatt\anaconda3\lib\site-packages\sklearn\neighbors\_base.py:413: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='manhattan',
                 metric_params=None, n_jobs=-1, n_neighbors=25, p=2,
                 radius=1.0)

In [64]:
# Тестируем что получилось 
# На выходе имеем веса фильмов и их id
test = change_string('Crime|Drama|Thriller')

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

res = neigh.kneighbors(X_tfidf2, return_distance=True)
res

(array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[5788, 6155, 6839, 7986, 4607,  442,  397, 4914, 7926, 6193, 7931,
         8282,   98, 6727, 8384, 8961, 5744, 5714, 8505, 2389, 8018, 4895,
         6795, 2909, 3028]], dtype=int64))

In [65]:
# узнаем, что за фильмы предложим пользователю 
movies.iloc[res[1][0]]

,movieId,title,genres
5788,31590,Hands Off the Loot (Touchez pas au grisbi) (1954),Crime|Drama|Thriller
6155,44199,Inside Man (2006),Crime|Drama|Thriller
6839,61352,Traitor (2008),Crime|Drama|Thriller
7986,96811,End of Watch (2012),Crime|Drama|Thriller
4607,6862,Out of Time (2003),Crime|Drama|Thriller
442,507,"Perfect World, A (1993)",Crime|Drama|Thriller
397,456,Fresh (1994),Crime|Drama|Thriller
4914,7368,Never Die Alone (2004),Crime|Drama|Thriller
7926,95508,Cleanskin (2012),Crime|Drama|Thriller
6193,45062,"Sentinel, The (2006)",Crime|Drama|Thriller
